In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime, os
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
from tqdm import tqdm_notebook
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split, KFold, GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

from utils import *

SEED = 42
seed_everything(SEED)

In [2]:
train = pd.read_pickle('../input/reduced/train_df_features.pkl')
test = pd.read_pickle('../input/reduced/test_df_features.pkl')
train_y = pd.read_pickle('../input/reduced/train_y.pkl')

In [3]:
print('training data size is ', train.shape)
print('training label size is', train_y.shape)
print('testing data size is', test.shape)

training data size is  (590540, 440)
training label size is (590540,)
testing data size is (506691, 440)


In [4]:
rm_cols = ['TransactionID', 'TransactionDT', 'isFraud', 'ProductCD']

In [5]:
features_columns = list(train)
for col in rm_cols:
    if col in features_columns:
        features_columns.remove(col)

In [6]:
########################### Model params
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.03,
                    'num_leaves': 2**8,
                    'max_depth':-1,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.7,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':500,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': 42,
                    'early_stopping_rounds':100, 
                } 

In [7]:
test_predictions, auc_score = make_predictions(train, test, train_y, features_columns, lgb_params)

Fold: 0  -  453219 137321
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	training's auc: 0.999809	valid_1's auc: 0.906948
Fold: 1  -  488908 101632
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	training's auc: 0.999837	valid_1's auc: 0.94113
Fold: 2  -  497955 92585
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	training's auc: 0.999806	valid_1's auc: 0.933304
Fold: 3  -  501214 89326
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	training's auc: 0.999737	valid_1's auc: 0.932974
Fold: 4  -  504519 86021
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[310]	training's auc: 0.997616	valid_1's auc: 0.943057
Fold: 5  -  506885 83655
Training until validation scores

In [9]:
# test_predictions.to_csv('../submissions/sub.csv',index=True) 